# ETL

In [94]:
# ETL
#Importing libraries

import pandas as pd 
import json 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ignac\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

#### To load and normalize the data into a dataframe from a nested json, using one column (flatten it) to extend into the dataframe:

    arguments:
    file = json file to load
    encode = encoding of said file
    path = column to be flattend to form the dataframe 
    depth = level to wich flaten said column, default is 0 (int)
    meta = other columns that should be added (unchanged)

In [95]:
def load_json_to_csv(file, encode='utf8', path="", depth=0, meta="", route="original_file_route.csv"):

    if route == "original_file_route.csv": route = file[:-4] + "csv"

    with open(file, encoding=encode) as file:

        data = json.load(file)

    df = pd.json_normalize(data, record_path=[path], max_level=depth, meta=meta)

    df.to_csv(route, header=df.shape[1], index=False)

    return df

In [96]:
#Loading user reviews: 

reviews = load_json_to_csv("databases/australian_user_reviews_fixed_v2.json",path="reviews", meta="user_id")

In [97]:
reviews.drop_duplicates(inplace=True)
reviews.reset_index(inplace=True)

In [98]:
#Loading user items: 

items_users = load_json_to_csv("databases/australian_users_items_fixed_v2.json",path="items", meta=["user_id",
                                                                                                    "steam_id"])

In [99]:
items_users.drop_duplicates(inplace=True)
items_users.reset_index(inplace=True)

In [100]:
#Loading games: 
#The 'games' df ends up with a lot of NaN so here we cleanse it
# None of the other df has NaNs in them

games = pd.read_json("databases/output_steam_games_fixed.json")

games.to_csv("databases/output_steam_games_fixed.csv", encoding='utf8')

In [101]:
games.dropna(inplace=True)
games.reset_index(inplace=True)


In [102]:
#Creating dummies for the genres in order to better work with them
games['genres'] = games['genres'].fillna('[]')
games['genres'] = games['genres'].apply(lambda x: ', '.join(x))

dummy_genres = games['genres'].str.get_dummies(', ')

games_with_dummies = pd.concat([games, dummy_genres], axis=1)

In [103]:
games_with_dummies.columns

Index(['index', 'publisher', 'genres', 'app_name', 'title', 'url',
       'release_date', 'tags', 'reviews_url', 'specs', 'price', 'early_access',
       'id', 'developer', 'Accounting', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'N', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing', 'a'],
      dtype='object')

In [104]:
games_with_dummies.drop(inplace=True, columns=['early_access', 
                                                'specs', 
                                                'reviews_url',
                                                'early_access', 
                                                'app_name', 
                                                'index'])

### Sentiment analysis 

Based on the written reviews a sentiment analysis is made to each of them in order 
to get a numerical column that represents the review, where 0 is for a negative opinion
and 2 for positive, having 1 for neutral when there isnt a review 

In [105]:
#Because the most common words are already known thanks to the EDA, the ones that are bastly+
#used and aport little information can be ignored in the analysis


# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.5:
            return 2  # Good score
        elif compound_score <= -0.5:
            return 0  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values


# Convertir la columna 'review' a tipo de dato str
reviews['review'] = reviews['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review'
reviews['sentiment_score'] = reviews['review'].apply(get_sentiment_score)


In [106]:
reviews.columns

Index(['index', 'funny', 'posted', 'last_edited', 'item_id', 'helpful',
       'recommend', 'review', 'user_id', 'sentiment_score'],
      dtype='object')

In [107]:
reviews.drop(columns=["index","review","last_edited"], inplace=True)

In [108]:
#Given that for this proyect is only going to be needed the year of any review
#the column is changed in order to just keep that

reviews['posted_year'] = reviews['posted'].str.extract(r'(\d{4})')

reviews.drop('posted' , axis = 1, inplace = True)

In [109]:
items_users.columns

Index(['index', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id', 'steam_id'],
      dtype='object')

In [110]:
items_users.drop(columns=["index"], inplace=True)

In [111]:
# Droping useless columns:
games_with_dummies.columns

Index(['publisher', 'genres', 'title', 'url', 'release_date', 'tags', 'price',
       'id', 'developer', 'Accounting', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'N', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing', 'a'],
      dtype='object')

In [112]:
games_with_dummies.drop(columns=["url"], inplace=True)

In [113]:
games_with_dummies.rename({"item_id":"id"},inplace=True)

In [114]:
games_with_dummies['release_year'] = games_with_dummies['release_date'].str.extract(r'(\d{4})')
games_with_dummies['release_year'] = games_with_dummies['release_year'].astype(float)


games_with_dummies.drop('release_date' , axis = 1, inplace = True)



Joining all dataframes on 'userid' in order to have a unified database:

In [115]:
games_with_dummies.columns

Index(['publisher', 'genres', 'title', 'tags', 'price', 'id', 'developer',
       'Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'N', 'Photo Editing', 'RPG', 'Racing',
       'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing', 'a', 'release_year'],
      dtype='object')

In [116]:
#In order to merge the games df its needed to change its "id" column dtype

games_with_dummies["item_id"] = games_with_dummies["id"].astype("object")
games_with_dummies["item_id"] = games_with_dummies["item_id"].apply(lambda x: str(int(x)) )

In [117]:
all_df = pd.merge(games_with_dummies, items_users,on="item_id")

all_df.shape

(2531919, 38)

In [118]:
all_df = pd.merge(items_users, reviews, on=["user_id", "item_id"])

In [119]:
all_df = pd.merge(all_df, games_with_dummies, on="item_id")

In [120]:
all_df.shape



(24752, 43)

In [121]:
all_df.to_csv("databases/all_data.csv", sep=",")

In [124]:
games_with_dummies["item_id"].head(10)

0    761140
1    643980
2    670290
3    767400
4    773570
5    772540
6    774276
7    774277
8    774278
9    768800
Name: item_id, dtype: object